In [ ]:
import pandas as pd
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

df = pd.read_csv('mvs.csv')
df.isnull().sum()  # checking for missing values
df = df.head(500)  # reduce dataset size for now

df['fullplot'] = df['fullplot'].fillna('')  # Handle missing values
df['fullplot'] = df['fullplot'].apply(lambda x: x.lower())  # Convert to lowercase
df['fullplot'] = df['fullplot'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))  # Remove punctuation

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['fullplot'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend_movies(movie_title, df, cosine_sim):
    movie_title = movie_title.lower()
    if movie_title not in df['title'].str.lower().values:
        return "Movie not found!"

    index = df[df['title'].str.lower() == movie_title].index[0]
    similar_movies = list(enumerate(cosine_sim[index]))
    similar_movies = sorted(similar_movies, key=lambda x: x[1], reverse=True)[1:6]
    recommended_movies = [df.iloc[i[0]]['title'] for i in similar_movies]
    return recommended_movies

print(recommend_movies("Carmencita", df, cosine_sim))

